# 1. 기상청 API 데이터 수집

In [ ]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta

# 기상청 API 데이터 수집 (2022년 1월 1일 ~ 2024년 12월 31일)
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 12, 31)
date_list = [(start_date + timedelta(days=i)).strftime('%Y%m%d') for i in range((end_date - start_date).days + 1)]

weather_records = []
col_names = [f'col{i}' for i in range(56)]  # 데이터 열 개수에 맞춤
url_weather = "https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php"

for date in date_list:
    params_weather = {
        "tm1": date,
        "tm2": date,
        "stn": "108",
        "help": "1",
        "authKey": "n7wd-Im8T368HfiJvO9-5Q"  # 실제 인증키로 교체 필요
    }
    try:
        response_weather = requests.get(url_weather, params=params_weather, timeout=10)
        response_weather.encoding = 'euc-kr'
        lines = response_weather.text.split('\n')
        lines = [line for line in lines if line and not line.startswith('#')]
        if not lines:
            continue
        csv_data = '\n'.join(lines)
        df_w = pd.read_csv(StringIO(csv_data), sep=r'\s+', header=None, names=col_names, engine='python')
        df_w = df_w.reset_index(drop=True)
        df_w['date'] = df_w['col0'].astype(str).str[:8]
        df_w['date_dt'] = pd.to_datetime(df_w['date'], format='%Y%m%d', errors='coerce')
        # 주요 변수만 추출
        df_w = df_w[['date_dt', 'col16', 'col21', 'col2']]
        df_w.columns = ['date_dt', 'TA_AVG', 'HM_AVG', 'WS_AVG']
        weather_records.append(df_w)
    except Exception as e:
        continue

if weather_records:
    df_weather_all = pd.concat(weather_records, ignore_index=True)
else:
    df_weather_all = pd.DataFrame()

print('기상청 데이터 수집 완료, 샘플 데이터:')
print(df_weather_all.head())
print('총 수집된 날짜 수:', df_weather_all['date_dt'].nunique())

# 2. 산불위험예보 목록정보 전처리

In [ ]:
df_risk = pd.read_csv('산림청 국립산림과학원_대형산불위험예보목록정보_20250430.csv', encoding='euc-kr')
df_risk = df_risk.rename(columns={
    '예보일시': 'date',
    '시도명': 'province',
    '시군구명': 'city',
    '실효습도': 'effective_humidity',
    '풍속': 'wind_speed',
    '등급': 'risk_grade'
})
df_risk['date_dt'] = pd.to_datetime(df_risk['date'], errors='coerce').dt.date
df_risk['effective_humidity'] = pd.to_numeric(df_risk['effective_humidity'], errors='coerce')
df_risk['wind_speed'] = pd.to_numeric(df_risk['wind_speed'], errors='coerce')

# 2022~2024년 데이터만 필터링
df_risk = df_risk[
    (df_risk['date_dt'] >= pd.to_datetime('2022-01-01').date()) &
    (df_risk['date_dt'] <= pd.to_datetime('2024-12-31').date())
]

In [ ]:
df_fire = pd.read_csv('sanbul.csv', encoding='cp949')
df_fire = df_fire.rename(columns={
    '발생일시_년': 'year',
    '발생일시_월': 'month',
    '발생일시_일': 'day',
    '발생장소_시도': 'province',
    '발생장소_시군구': 'city'
})
df_fire['date_dt'] = pd.to_datetime(
    df_fire['year'].astype(str) + '-' +
    df_fire['month'].astype(str).str.zfill(2) + '-' +
    df_fire['day'].astype(str).str.zfill(2),
    errors='coerce'
).dt.date

# 2022~2024년 데이터만 필터링
df_fire = df_fire[
    (df_fire['date_dt'] >= pd.to_datetime('2022-01-01').date()) &
    (df_fire['date_dt'] <= pd.to_datetime('2024-12-31').date())
]

# 일별, 지역별 산불 발생 건수 및 유무 집계
fire_group = df_fire.groupby(['date_dt', 'province', 'city']).size().reset_index(name='fire_count')
fire_group['fire_occurred'] = (fire_group['fire_count'] > 0).astype(int)

In [ ]:
# 1차 병합 (날짜 기준)
df_weather_all['date_dt'] = pd.to_datetime(df_weather_all['date_dt']).dt.date
risk_weather = pd.merge(df_risk, df_weather_all, on='date_dt', how='inner')

print('1차 병합(기상+위험예보) 샘플:', risk_weather.head())

# 2차 병합 (날짜+지역 기준)
final_df = pd.merge(
    risk_weather,
    fire_group,
    on=['date_dt', 'province', 'city'],
    how='left'
)

# 결측치 처리: 산불이 발생하지 않은 경우 0으로 채움
final_df['fire_occurred'] = final_df['fire_occurred'].fillna(0).astype(int)
final_df['fire_count'] = final_df['fire_count'].fillna(0).astype(int)

# feature 결측치 제거
features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed']
final_df = final_df.dropna(subset=features)

print('타겟 변수 분포:', final_df['fire_occurred'].value_counts())

In [ ]:
from imblearn.over_sampling import SMOTE

# SMOTE 적용 (0과 1이 모두 있을 때만 실행)
X = final_df[features].astype(float)
y = final_df['fire_occurred']

minority_count = sum(y == 1)

if minority_count < 2:
    print("소수 클래스 샘플 부족으로 SMOTE 적용 불가. 원본 데이터 사용")
    X_resampled, y_resampled = X, y
else:
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 클래스 비율(ratio) 계산
ratio = (y_resampled == 0).sum() / (y_resampled == 1).sum()

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled,
    test_size=0.2,
    random_state=42,
    stratify=y_resampled
)

# 모델 학습
model = XGBClassifier(scale_pos_weight=ratio, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
preds = model.predict(X_test)
print("정확도:", accuracy_score(y_test, preds))
print("\n분류 리포트:")
print(classification_report(y_test, preds))
print("\n혼동 행렬:")
print(confusion_matrix(y_test, preds))

# 예측 결과 표로 확인
result_df = X_test.copy()
result_df['actual_fire_occurred'] = y_test
result_df['predicted_fire_occurred'] = preds
print(result_df.head())

In [ ]:
# 모델 저장 (XGBoost 네이티브 방식)
model.save_model('xgb_fire_model_smote.json')

# 저장 확인
import os
print('모델 저장 여부:', os.path.exists('xgb_fire_model_smote.json'))